In [2]:
import numpy as np
import cv2, os, pyautogui, time
from PIL import Image
import matplotlib.pyplot as plt
import warnings

from src.face_features import *
from src.tools import *

width, height= pyautogui.size()

In [3]:
from keras.layers import *
from keras.models import Model
from src.model_config import *

img_size = 64
n_channel = 1
mask_size = 25

conv1_eye_size = 5
conv1_eye_out = 128
conv1_eye_strides = 1
pool1_eye_size = 2
pool1_eye_stride = 2

conv2_eye_size = 3
conv2_eye_out = 64
conv2_eye_strides = 2
pool2_eye_size = 2
pool2_eye_stride = 2

conv3_eye_size = 3
conv3_eye_out = 64
pool3_eye_size = 2
pool3_eye_stride = 2

conv1_face_size = 5
conv1_face_out = 128
conv1_face_strides = 1
pool1_face_size = 2
pool1_face_stride = 2

conv2_face_size = 3
conv2_face_out = 64
conv2_face_strides = 2
pool2_face_size = 2
pool2_face_stride = 2

conv3_face_size = 3
conv3_face_out = 64
pool3_face_size = 2
pool3_face_stride = 2

fc_eye_size = 128
fc_face_size = 128
fc2_face_size = 64
fc_face_mask_size = 128
fc2_face_mask_size = 128
fc_size = 256
fc2_size = 2


right_input = Input(shape=(img_size, img_size, n_channel))
left_input = Input(shape=(img_size, img_size, n_channel))
face_input = Input(shape=(img_size, img_size, n_channel))
mask_input = Input(shape=(mask_size * mask_size,))

conv1_right = Conv2D(conv1_eye_out, kernel_size=conv1_eye_size, strides=conv1_eye_strides)(right_input)
conv1_pool_right = MaxPooling2D(pool_size=pool1_eye_size, strides=pool1_eye_stride)(conv1_right)
conv2_right = Conv2D(conv2_eye_out, kernel_size=conv2_eye_size, strides=conv2_eye_strides)(conv1_pool_right)
conv2_pool_right = MaxPooling2D(pool_size=pool2_eye_size, strides=pool2_eye_stride)(conv2_right)
conv3_right = Conv2D(conv3_eye_out, kernel_size=conv3_eye_size)(conv2_pool_right)
conv3_pool_right = MaxPooling2D(pool_size=pool3_eye_size, strides=pool3_eye_stride)(conv3_right)

conv1_left = Conv2D(conv1_eye_out, kernel_size=conv1_eye_size, strides=conv1_eye_strides)(left_input)
conv1_pool_left = MaxPooling2D(pool_size=pool1_eye_size, strides=pool1_eye_stride)(conv1_left)
conv2_left = Conv2D(conv2_eye_out, kernel_size=conv2_eye_size, strides=conv2_eye_strides)(conv1_pool_left)
conv2_pool_left = MaxPooling2D(pool_size=pool2_eye_size, strides=pool2_eye_stride)(conv2_left)
conv3_left = Conv2D(conv3_eye_out, kernel_size=conv3_eye_size)(conv2_pool_left)
conv3_pool_left = MaxPooling2D(pool_size=pool3_eye_size, strides=pool3_eye_stride)(conv3_left)

conv1_face = Conv2D(conv1_face_out, kernel_size=conv1_face_size, strides=conv1_face_strides)(face_input)
conv1_pool_face = MaxPooling2D(pool_size=pool1_face_size, strides=pool1_face_stride)(conv1_face)
conv2_face = Conv2D(conv2_face_out, kernel_size=conv2_eye_size, strides=conv2_face_strides)(conv1_pool_face)
conv2_pool_face = MaxPooling2D(pool_size=pool2_face_size, strides=pool2_face_stride)(conv2_face)
conv3_face = Conv2D(conv3_face_out, kernel_size=conv3_eye_size)(conv2_pool_face)
conv3_pool_face = MaxPooling2D(pool_size=pool3_face_size, strides=pool3_face_stride)(conv3_face)

right_out = Reshape([-1, int(np.prod(conv3_pool_right.get_shape()[1:]))])(conv3_pool_right)
left_out = Reshape([-1, int(np.prod(conv3_pool_left.get_shape()[1:]))])(conv3_pool_left)

eyes = Concatenate(axis=-1)([right_out, left_out])
eyes = Dense(fc_eye_size, activation="relu")(eyes)
eyes = Flatten()(eyes)

face = Reshape([-1, int(np.prod(conv3_pool_face.get_shape()[1:]))])(conv3_pool_face)
face = Dense(fc_face_size, activation="relu")(face)
face = Dense(fc2_face_size, activation="relu")(face)
face = Flatten()(face)

face_mask = Dense(fc_face_mask_size, activation="relu")(mask_input)
face_mask = Dense(fc2_face_mask_size, activation="relu")(face_mask)

fc = Concatenate(axis=-1)([eyes, face, face_mask])
fc2 = Dense(fc_size, activation="relu")(fc)
out = Dense(fc2_size, activation="sigmoid")(fc2)

model = Model([left_input, right_input, face_input, mask_input], out)
model.compile(optimizer='adam', loss="mse")
model.summary() 

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 60, 60, 128)  3328        input_5[0][0]                    
_______________________________________________________________________________________

In [ ]:
from src.model import model

model.compile(optimizer='adam', loss="mse")
# model.summary() 

In [ ]:
# pickle.dump([X_left, X_right, X_face, X_mask, Y], open("save.pkl", "wb"))

# X_left = np.concatenate([X_left, X_left_old])
# X_right = np.concatenate([X_right, X_right_old])
# X_face = np.concatenate([X_face, X_face_old])
# X_mask = np.concatenate([X_mask, X_mask_old])
# Y = np.concatenate([Y, Y_old])

In [ ]:
X_left_old, X_right_old, X_face_old, X_mask_old, Y_old = X_left, X_right, X_face, X_mask, Y

In [5]:
import pickle

[X_left, X_right, X_face, X_mask, Y] = pickle.load(open("save.pkl", "rb"))
print(X_left.shape, X_right.shape, X_face.shape, X_mask.shape, Y.shape)

(9746, 64, 64, 1) (9746, 64, 64, 1) (9746, 64, 64, 1) (9746, 625) (9746, 2)


In [3]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(list(range(len(Y))), train_size=0.9, shuffle=True)

X_train = [X_left[train_set], X_right[train_set], X_face[train_set], X_mask[train_set]]
X_test = [X_left[test_set], X_right[test_set], X_face[test_set], X_mask[test_set]]

In [4]:
from keras.models import load_model
model = load_model("checkpoints")

In [22]:
history = model.fit(X_train, np.array(Y)[train_set], batch_size=256, epochs=200, validation_data=(X_test, np.array(Y)[test_set]))

Epoch 1/200
35/35 [==============================] - 8s 235ms/step - loss: 0.0472 - val_loss: 0.0375
Epoch 2/200
35/35 [==============================] - 7s 198ms/step - loss: 0.0353 - val_loss: 0.0342
Epoch 3/200
35/35 [==============================] - 7s 193ms/step - loss: 0.0313 - val_loss: 0.0283
Epoch 4/200
35/35 [==============================] - 7s 189ms/step - loss: 0.0264 - val_loss: 0.0238
Epoch 5/200
35/35 [==============================] - 7s 188ms/step - loss: 0.0224 - val_loss: 0.0236
Epoch 6/200
35/35 [==============================] - 7s 187ms/step - loss: 0.0176 - val_loss: 0.0168
Epoch 7/200
35/35 [==============================] - 7s 192ms/step - loss: 0.0134 - val_loss: 0.0123
Epoch 8/200
35/35 [==============================] - 7s 188ms/step - loss: 0.0103 - val_loss: 0.0088
Epoch 9/200
35/35 [==============================] - 7s 189ms/step - loss: 0.0081 - val_loss: 0.0082
Epoch 10/200
35/35 [==============================] - 7s 199ms/step - loss: 0.0073 - val_lo

KeyboardInterrupt: 

In [24]:
# Plot history: MAE
plt.figure(figsize=(10,6), dpi=80)
plt.plot(history.history['loss'], label='MSE (training data)')
plt.plot(history.history['val_loss'], label='MSE (validation data)')
plt.title('MSE over training')
plt.ylabel('MSE value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()

NameError: name 'history' is not defined

<Figure size 800x480 with 0 Axes>

In [ ]:
model.predict(X_train)

In [ ]:
model.save("checkpoints")